# Capítulo 18. Aprendizaje de refuerzo

El aprendizaje por refuerzo (RL) es uno de los campos más emocionantes del aprendizaje automático en la actualidad, y también uno de los más antiguos. Ha existido desde la década de 1950, produciendo muchas aplicaciones interesantes a lo largo de los años,  particularmente en juegos (por ejemplo, TD-Gammon, un programa de juego de Backgammon) y en el control de máquinas, pero rara vez aparece en los titulares. Sin embargo, se produjo una revolución en 2013, cuando los investigadores de una startup británica llamada DeepMind demostraron un sistema que podía aprender a jugar casi cualquier juego de Atari desde cero,⁠ finalmente superando a los humanos⁠ en la mayoría de ellos, usando solo píxeles en bruto como entradas y sin ningún conocimiento previo de las reglas de los juegos.⁠4 Esta fue la primera de una serie de hazañas increíbles, que culminó con la victoria de su sistema AlphaGo contra Lee Sedol, un legendario jugador profesional del juego de Go, en marzo de 2016 y contra Ke Jie, el campeón del mundo, en mayo de 2017. Ningún programa se había acercado a vencer a un maestro de este juego, y mucho menos al campeón del mundo. Hoy en día, todo el campo de RL está hirviendo con nuevas ideas, con una amplia gama de aplicaciones.

Entonces, ¿cómo logró DeepMind (comprado por Google por más de 500 millones de dólares en 2014)? En retrospectiva, parece bastante simple: aplicaron el poder del aprendizaje profundo al campo del aprendizaje por refuerzo, y funcionó más allá de sus sueños más salvajes. En este capítulo, primero explicaré qué es el aprendizaje por refuerzo y en qué es bueno, y luego presentaré dos de las técnicas más importantes en el aprendizaje por refuerzo profundo: los gradientes de políticas y las redes Q profundas, incluida una discusión sobre los procesos de decisión de Markov. ¡Empecemos!

# Aprender a optimizar las recompensas

En el aprendizaje por refuerzo, un agente de software hace observaciones y toma acciones dentro de un entorno, y a cambio recibe recompensas del entorno. Su objetivo es aprender a actuar de una manera que maximice sus recompensas esperadas con el tiempo. Si no te importa un poco el antropomorfismo, puedes pensar en las recompensas positivas como placer y las recompensas negativas como dolor (el término "recompensa" es un poco engañoso en este caso). En resumen, el agente actúa en el entorno y aprende por ensayo y error para maximizar su placer y minimizar su dolor.

Este es un entorno bastante amplio, que puede aplicarse a una amplia variedad de tareas. Aquí hay algunos ejemplos (ver Figura 18-1):

1. l agente puede ser el programa que controla un robot. En este caso, el entorno es el mundo real, el agente observa el medio ambiente a través de un conjunto de sensores como cámaras y sensores táctiles, y sus acciones consisten en enviar señales para activar los motores. Puede programarse para obtener recompensas positivas cada vez que se acerca al destino objetivo, y recompensas negativas cada vez que pierde el tiempo o va en la dirección equivocada.

2. El agente puede ser el programa que controla a la Sra. Pac-Man. En este caso, el entorno es una simulación del juego de Atari, las acciones son las nueve posiciones posibles del joystick (arriba a la izquierda, abajo, centro, etc.), las observaciones son capturas de pantalla y las recompensas son solo los puntos del juego.

3. Del mismo modo, el agente puede ser el programa jugando a un juego de mesa como Go. Solo recibe una recompensa si gana.

4. El agente no tiene que controlar una cosa en movimiento física (o virtualmente). Por ejemplo, puede ser un termostato inteligente, obteniendo recompensas positivas cada vez que está cerca de la temperatura objetivo y ahorra energía, y recompensas negativas cuando los humanos necesitan ajustar la temperatura, por lo que el agente debe aprender a anticipar las necesidades humanas.

5. El agente puede observar los precios del mercado de valores y decidir cuánto comprar o vender cada segundo. Las recompensas son, obviamente, las ganancias y pérdidas monetarias.

Tenga en cuenta que puede que no haya ninguna recompensa positiva en absoluto; por ejemplo, el agente puede moverse en un laberinto, obteniendo una recompensa negativa en cada paso, ¡así que es mejor que encuentre la salida lo más rápido posible! Hay muchos otros ejemplos de tareas para las que el aprendizaje por refuerzo es adecuado, como los coches autónomos, los sistemas de recomendación, la colocación de anuncios en una página web o el control de dónde debe centrar su atención un sistema de clasificación de imágenes.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1801.png)

(_Figura 18-1. Ejemplos de aprendizaje de refuerzo: (a) robótica, (b) Sra. Pac-Man, (c) Go player, (d) termostato, (e) operador automático⁠_)


# Búsqueda de políticas


El algoritmo que un agente de software utiliza para determinar sus acciones se llama su política. La política podría ser una red neuronal que toma las observaciones como entradas y produce la acción a tomar (ver Figura 18-2).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1802.png)

(_Figura 18-2. Aprendizaje de refuerzo utilizando una política de red neuronal_)

La política puede ser cualquier algoritmo que se te ocurra, y no tiene por que ser determinista. De hecho, ¡en algunos casos ni siquiera tiene que observar el medio ambiente! Por ejemplo, considere una aspiradora robótica cuya recompensa es la cantidad de polvo que recoge en 30 minutos. Su política podría ser avanzar con alguna probabilidad p cada segundo, o rotar al azar a la izquierda o a la derecha con probabilidad 1 - p. El ángulo de rotación sería un ángulo aleatorio entre -r y +r. Dado que esta política implica cierta aleatoriedad, se llama política estocástica. El robot tendrá una trayectoria errática, lo que garantiza que eventualmente llegará a cualquier lugar al que pueda llegar y recoger todo el polvo. La pregunta es, ¿cuánto polvo recogerá en 30 minutos?

¿Cómo entrenarías a un robot así? Solo hay dos parámetros de política que puedes ajustar: la probabilidad p y el rango de ángulo r. Un posible algoritmo de aprendizaje podría ser probar muchos valores diferentes para estos parámetros y elegir la combinación que funcione mejor (ver Figura 18-3). Este es un ejemplo de búsqueda de políticas, en este caso utilizando un enfoque de fuerza bruta. Cuando el espacio de la política es demasiado grande (que generalmente es el caso), encontrar un buen conjunto de parámetros de esta manera es como buscar una aguja en un pajar gigantesco.

Otra forma de explorar el espacio de políticas es utilizar algoritmos genéticos. Por ejemplo, podrías crear al azar una primera generación de 100 políticas y probarlas, luego "matar" las 80 peores políticas⁠ y hacer que los 20 sobrevivientes produzcan 4 hijos cada uno. Una descendencia es una copia de su padre⁠ más alguna variación aleatoria. Las políticas sobrevivientes más sus hijos juntos constituyen la segunda generación. Puedes continuar iterando a través de generaciones de esta manera hasta que encuentres una buena política.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1803.png)

(_Figura 18-3. Cuatro puntos en el espacio de política (izquierda) y el comportamiento correspondiente del agente (derecha)_)

Otro enfoque es utilizar técnicas de optimización, evaluando los gradientes de las recompensas con respecto a los parámetros de la política, y luego ajustando estos parámetros siguiendo los gradientes hacia recompensas más altas. ⁠Discutiremos este enfoque, llamado gradientes de política (PG), con más detalle más adelante en este capítulo. Volviendo al robot aspirador, podría aumentar ligeramente p y evaluar si hacerlo aumenta la cantidad de polvo recogido por el robot en 30 minutos; si lo hace, entonces aumentar p un poco más, o de lo contrario reducir p. Implementaremos un popular algoritmo de PG usando TensorFlow, pero antes de hacerlo, necesitamos crear un entorno para que el agente viva⁠, así que es hora de introducir OpenAI Gym.

# Introducción al gimnasio OpenAI

Uno de los desafíos del aprendizaje por refuerzo es que para capacitar a un agente, primero necesitas tener un entorno de trabajo. Si quieres programar un agente que aprenda a jugar a un juego de Atari, necesitarás un simulador de juegos de Atari. Si quieres programar un robot andante, entonces el entorno es el mundo real, y puedes entrenar directamente a tu robot en ese entorno. Sin embargo, esto tiene sus límites: si el robot se cae de un precipicio, no puedes simplemente hacer clic en Deshacer. Tampoco se puede acelerar el tiempo, agregar más potencia de cálculo no hará que el robot se mueva más rápido, y generalmente es demasiado caro entrenar a 1.000 robots en paralelo. En resumen, el entrenamiento es difícil y lento en el mundo real, por lo que generalmente se necesita un entorno simulado al menos para el entrenamiento de arranque. Por ejemplo, podrías usar una biblioteca como PyBullet o MuJoCo para la simulación de física 3D.

OpenAI Gym⁠ es un conjunto de herramientas que proporciona una amplia variedad de entornos simulados (juegos Atari, juegos de mesa, simulaciones físicas 2D y 3D, etc.), que puede utilizar para entrenar a agentes, compararlos o desarrollar nuevos algoritmos RL.

OpenAI Gym está preinstalado en Colab, pero es una versión más antigua, por lo que tendrás que reemplazarla por la más reciente. También necesitas instalar algunas de sus dependencias. Si está codificando en su propia máquina en lugar de Colab, y siguió las instrucciones de instalación en https://homl.info/install, puede omitir este paso; de lo contrario, introduzca estos comandos:

In [ ]:
# Only run these commands on Colab or Kaggle!
%pipinstalar -q -U gimnasio%pipinstalar -q -U gym[classic_control,box2d,atari,accept-rom-license]

El primer comando `%pip` actualiza Gym a la última versión. La opción `-q` significa silencio: hace que la salida sea menos detallada. La opción `-U` significa actualización. El segundo comando `%pip` instala las bibliotecas necesarias para ejecutar varios tipos de entornos. Esto incluye entornos clásicos de la teoría del control (la ciencia de controlar sistemas dinámicos), como equilibrar un poste en un carro. También incluye entornos basados en la biblioteca Box2D, un motor de física 2D para juegos. Por último, incluye entornos basados en Arcade Learning Environment (ALE), que es un emulador de juegos de Atari 2600. Varias ROM de juegos de Atari se descargan automáticamente y, al ejecutar este código, acepta las licencias de ROM de Atari.

Con eso, estás listo para usar OpenAI Gym. Vamos a importarlo y hacer un entorno:

In [ ]:
import gym

env = gym.make("CartPole-v1", render_mode="rgb_array")

Aquí, hemos creado un entorno CartPole. Esta es una simulación 2D en la que un carro se puede acelerar a la izquierda o a la derecha para equilibrar un poste colocado encima de él (ver Figura 18-4). Esta es una tarea de control clásica.

#### TIP

El diccionario `gym.envs.registry` contiene los nombres y especificaciones de todos los entornos disponibles.

#### -------------------------------------------------------------------------------------------------------------

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1804.png)

(_Figura 18-4. El entorno de CartPole_)

Una vez creado el entorno, debe inicializarlo utilizando el método `reset()`, especificando opcionalmente una semilla aleatoria. Esto devuelve la primera observación. Las observaciones dependen del tipo de entorno. Para el entorno CartPole, cada observación es una matriz NumPy 1D que contiene cuatro flotadores que representan la posición horizontal del carro (`0.0` = centro), su velocidad (positivo significa derecha), el ángulo del poste (`0.0` = vertical) y su velocidad angular ( positivo significa en el sentido de las agujas del reloj). El método `reset()` también devuelve un diccionario que puede contener información adicional específica del entorno. Esto puede resultar útil para depurar o para entrenar. Por ejemplo, en muchos entornos Atari, contiene el número de vidas que quedan. Sin embargo, en el entorno CartPole, este diccionario está vacío.

In [ ]:
obs, info = env.reset(seed=42)
obs

# array([ 0.0273956 , -0.00611216,  0.03585979,  0.0197368 ], dtype=float32)

info

#{}

Llamemos al método `render()` para representar este entorno como una imagen. Dado que configuramos `render_mode="rgb_array"` al crear el entorno, la imagen se devolverá como una matriz NumPy:

In [ ]:
img = env.render()
img.shape  # height, width, channels (3 = Red, Green, Blue)

#(400, 600, 3)

Luego puede usar la función `imshow()` de Matplotlib para mostrar esta imagen, como de costumbre.

Ahora preguntemos al medio ambiente qué acciones son posibles:

In [ ]:
env.action_space

#Discrete(2)

`Discrete(2)` significa que las acciones posibles son los enteros 0 y 1, que representan la aceleración a la izquierda o a la derecha. Otros entornos pueden tener acciones discretas adicionales, u otros tipos de acciones (por ejemplo, continuas). Dado que el poste se inclina hacia la derecha (`obs[2] > 0`), aceleremos el carro hacia la derecha:

In [1]:
action = 1  # accelerate right
obs, reward, done, truncated, info = env.step(action)
obs
#array([ 0.02727336,  0.18847767,  0.03625453, -0.26141977], dtype=float32)

reward
#1.0

done
#False

truncated
#False

info
#{}

SyntaxError: invalid syntax (2503881744.py, line 5)

El método `step()` ejecuta la acción deseada y retorna 5 valores:

**obs**

    Esta es la nueva observación. El carrito ahora se está moviendo hacia la derecha (`obs[1] > 0`). El polo todavía está inclinado hacia la derecha (`obs[2] > 0`), pero su velocidad angular es ahora negativa (`obs[3] < 0`), por lo que es probable que se incline hacia la izquierda después del siguiente paso.

**reward**

    En este entorno, obtienes una recompensa de 1,0 en cada paso, sin importar lo que hagas, por lo que el objetivo es mantener el episodio funcionando el mayor tiempo posible.

**done**

    Este valor será `True` cuando finalice el episodio. Esto sucederá cuando el poste se incline demasiado, o se salga de la pantalla, o después de 200 pasos (en este último caso, has ganado). Después de eso, el entorno debe restablecerse antes de poder volver a utilizarlo.

**truncated**

    Este valor será `True` cuando un episodio se interrumpa antes de tiempo, por ejemplo, por un contenedor de entorno que impone un número máximo de pasos por episodio (consulte la documentación de Gym para obtener más detalles sobre los contenedores de entorno). Algunos algoritmos de RL tratan los episodios truncados de manera diferente a los episodios finalizados normalmente (es decir, cuando `done` es `True`), pero en este capítulo los trataremos de manera idéntica.

**info**

    Este diccionario específico del entorno puede proporcionar información adicional, al igual que la devuelta por el método `reset()`.
    
    
#### TIP

Una vez que haya terminado de usar un entorno, debe llamar a su método `close()` para liberar recursos.

#### -----------------------------------------------------------------------------------------------------------

Vamos a codificar una política simple que acelera hacia la izquierda cuando el poste se inclina hacia la izquierda y acelera hacia la derecha cuando el poste se inclina hacia la derecha. Llevaremos a car esta política para ver las recompensas promedio que obtiene en más de 500 episodios:

In [ ]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

totals = []
for episode in range(500):
    episode_rewards = 0
    obs, info = env.reset(seed=episode)
    for step in range(200):
        action = basic_policy(obs)
        obs, reward, done, truncated, info = env.step(action)
        episode_rewards += reward
        if done or truncated:
            break

    totals.append(episode_rewards)

Este código se explica por sí mismo. Echemos un vistazo al resultado:

In [ ]:
import numpy as np
np.mean(totals), np.std(totals), min(totals), max(totals)

#(41.698, 8.389445512070509, 24.0, 63.0)

Incluso con 500 intentos, esta política nunca logró mantener el poste en posición vertical durante más de 63 pasos consecutivos. No es genial. Si miras la simulación en el cuaderno de este capítulo, verás que el carro oscila a la izquierda y a la derecha cada vez con más fuerza hasta que el poste se inclina demasiado. Veamos si una red neuronal puede llegar a una mejor política.


## Políticas de redes neuronales


Vamos a crear una política de redes neuronales. Esta red neuronal tomará una observación como entrada, y producirá la acción que se va a ejecutar, al igual que la política que codificamos anteriormente. Más precisamente, estimará una probabilidad para cada acción, y luego seleccionaremos una acción al azar, de acuerdo con las probabilidades estimadas (ver Figura 18-5). En el caso del entorno CartPole, solo hay dos acciones posibles (izquierda o derecha), por lo que solo necesitamos una neurona de salida. Emprenderá la probabilidad p de la acción 0 (izquierda) y, por supuesto, la probabilidad de la acción 1 (derecha) será 1 - p. Por ejemplo, si genera 0,7, entonces elegiremos la acción 0 con una probabilidad del 70%, o la acción 1 con una probabilidad del 30%.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1805.png)

(_Figura 18-5. Política de redes neuronales_)

Puede que te preguntes por qué estamos eligiendo una acción aleatoria basada en las probabilidades dadas por la red neuronal, en lugar de simplemente elegir la acción con la puntuación más alta. Este enfoque permite al agente encontrar el equilibrio adecuado entre explorar nuevas acciones y explotar las acciones que se sabe que funcionan bien. Aquí hay una analogía: supongamos que vas a un restaurante por primera vez, y todos los platos se ven igual de atractivos, así que eliges uno al azar. Si resulta ser bueno, puedes aumentar la probabilidad de que lo pidas la próxima vez, pero no deberías aumentar esa probabilidad hasta el 100 %, o de lo contrario nunca probarás los otros platos, algunos de los cuales pueden ser incluso mejores que el que probaste. Este dilema de exploración/explotación es fundamental en el aprendizaje por refuerzo.

También tenga en cuenta que en este entorno en particular, las acciones y observaciones pasadas se pueden ignorar de forma segura, ya que cada observación contiene el estado completo del entorno. Si hubiera algún estado oculto, entonces es posible que también tengas que considerar acciones y observaciones pasadas. Por ejemplo, si el entorno solo revelara la posición del carro, pero no su velocidad, tendría que considerar no solo la observación actual, sino también la observación anterior para estimar la velocidad actual. Otro ejemplo es cuando las observaciones son ruidosas; en ese caso, generalmente desea utilizar las últimas observaciones para estimar el estado actual más probable. Por lo tanto, el problema de CartPole es tan simple como puede ser; las observaciones están libres de ruido y contienen el estado completo del medio ambiente.

Aquí está el código para construir una política básica de red neuronal usando Keras:

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(5, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

Utilizamos un modelo `Sequential` para definir la red de políticas. El número de entradas es el tamaño del espacio de observación (que en el caso de CartPole es 4) y solo tenemos cinco unidades ocultas porque es una tarea bastante simple. Finalmente, queremos generar una probabilidad única (la probabilidad de ir a la izquierda), por lo que tenemos una única neurona de salida que utiliza la función de activación sigmoidea. Si hubiera más de dos acciones posibles, habría una neurona de salida por acción y en su lugar usaríamos la función de activación softmax.

De acuerdo, ahora tenemos una política de red neuronal que tomará observaciones y producirá probabilidades de acción. Pero, ¿cómo lo entrenamos?


# Evaluación de acciones: El problema de la asignación de crédito


Si supiéramos cuál era la mejor acción en cada paso, podríamos entrenar la red neuronal como de costumbre, minimizando la entropía cruzada entre la distribución de probabilidad estimada y la distribución de probabilidad objetivo. Solo sería un aprendizaje supervisado regular. Sin embargo, en el aprendizaje de refuerzo, la única orientación que recibe el agente es a través de las recompensas, y las recompensas suelen ser escasas y retrasadas. Por ejemplo, si el agente logra equilibrar el poste durante 100 pasos, ¿cómo puede saber cuáles de las 100 acciones que tomó fueron buenas y cuáles fueron malas? Todo lo que sabe es que el poste cayó después de la última acción, pero seguramente esta última acción no es del todo responsable. Esto se llama el problema de la asignación de crédito: cuando el agente recibe una recompensa, es difícil para él saber qué acciones deben ser acreditadas (o culpadas) por ello. Piensa en un perro que es recompensado horas después de que se comporte bien; ¿entenderá por qué está siendo recompensado?

Para abordar este problema, una estrategia común es evaluar una acción basada en la suma de todas las recompensas que vienen después, generalmente aplicando un factor de descuento, γ (gamma), en cada paso. Esta suma de recompensas con descuento se llama el retorno de la acción. Considere el ejemplo de la Figura 18-6. Si un agente decide ir a la derecha tres veces seguidas y obtiene +10 de recompensa después del primer paso, 0 después del segundo paso y finalmente -50 después del tercer paso, entonces, suponiendo que usemos un factor de descuento γ = 0,8, la primera acción tendrá un retorno de 10 + γ × 0 + γ2 × (-50) = -22. Si el factor de descuento está cerca de 0, entonces las recompensas futuras no contarán mucho en comparación con las recompensas inmediatas. Por el contrario, si el factor de descuento está cerca de 1, entonces las recompensas en el futuro contarán casi tanto como las recompensas inmediatas. Los factores de descuento típicos varían de 0,9 a 0,99. Con un factor de descuento de 0,95, las recompensas de 13 pasos en el futuro cuentan aproximadamente la mitad que las recompensas inmediatas (desde 0,9513 ≈ 0,5), mientras que con un factor de descuento de 0,99, las recompensas de 69 pasos en el futuro cuentan para la mitad que las recompensas inmediatas. En el entorno de CartPole, las acciones tienen efectos a corto plazo, por lo que elegir un factor de descuento de 0,95 parece razonable.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1806.png)

(_Figura 18-6. Calcular el rendimiento de una acción: la suma de recompensas futuras con descuento_)

Por supuesto, una buena acción puede ir seguida de varias acciones malas que hacen que el poste caiga rápidamente, lo que resulta en que la buena acción tenga un bajo rendimiento. Del mismo modo, un buen actor a veces puede protagonizar una película terrible. Sin embargo, si jugamos el juego suficientes veces, en promedio, las acciones buenas obtendrán un mayor rendimiento que las malas. Queremos estimar cuánto mejor o peor es una acción, en comparación con las otras acciones posibles, en promedio. Esto se llama la ventaja de acción. Para esto, debemos ejecutar muchos episodios y normalizar todos los rendimientos de la acción, restando la media y dividiendo por la desviación estándar. Después de eso, podemos asumir razonablemente que las acciones con una ventaja negativa fueron malas, mientras que las acciones con una ventaja positiva fueron buenas. De acuerdo, ahora que tenemos una manera de evaluar cada acción, estamos listos para entrenar a nuestro primer agente usando gradientes de políticas. A ver cómo.

# Gradientes de políticas

Como se discutió anteriormente, los algoritmos PG optimizan los parámetros de una política siguiendo los gradientes hacia recompensas más altas. Una clase popular de algoritmos PG, llamada algoritmos REINFORCE, fue introducida en 1992⁠ por Ronald Williams. Aquí hay una variante común:

1. En primer lugar, deje que la política de la red neuronal juegue el juego varias veces y, en cada paso, calcule los gradientes que harían que la acción elegida fuera aún más probable, pero no aplique estos gradientes todavía.

2. Una vez que haya ejecutado varios episodios, calcule la ventaja de cada acción, utilizando el método descrito en la sección anterior.

3. Si la ventaja de una acción es positiva, significa que la acción probablemente fue buena, y desea aplicar los gradientes calculados anteriormente para que la acción sea aún más probable que se elija en el futuro. Sin embargo, si la ventaja de la acción es negativa, significa que la acción probablemente fue mala, y quieres aplicar los gradientes opuestos para que esta acción sea un poco menos probable en el futuro. La solución es multiplicar cada vector de gradiente por la ventaja de la acción correspondiente.

4. Finalmente, calcule la media de todos los vectores de gradiente resultantes y utilícela para realizar un paso de descenso de gradiente.

Usemos Keras para implementar este algoritmo. Entrenaremos la política de red neuronal que construimos anteriormente para que aprenda a equilibrar el polo en el carrito. En primer lugar, necesitamos una función que juegue un paso. Por ahora, fingiremos que cualquier acción que tome es la correcta para que podamos calcular la pérdida y sus gradientes. Estos gradientes solo se guardarán por un tiempo, y los modificaremos más tarde dependiendo de lo buena o mala que resultó ser la acción:

In [ ]:
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        left_proba = model(obs[np.newaxis])
        action = (tf.random.uniform([1, 1]) > left_proba)
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba))

    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, truncated, info = env.step(int(action))
    return obs, reward, done, truncated, grads

Vamos a caminar a través de esta función:

* Dentro del bloque `GradientTape` (ver Capítulo 12), comenzamos llamando al modelo, dándole una sola observación. Reformamos la observación para que se convierta en un lote que contenga una sola instancia, ya que el modelo espera un lote. Esto genera la probabilidad de ir a la izquierda.

- A continuación, tomamos una muestra de un flotante aleatorio entre 0 y 1 y comprobamos si es mayor que `left_proba`. La acción será `False` con probabilidad `left_proba`, o `True` con probabilidad `1 – left_proba`. Una vez que convertimos este booleano en un número entero, la acción será 0 (izquierda) o 1 (derecha) con las probabilidades apropiadas.

* Ahora definimos la probabilidad objetivo de ir a la izquierda: es 1 menos la acción (lanzar a un flotador). Si la acción es 0 (izquierda), entonces la probabilidad objetivo de ir a la izquierda será 1. Si la acción es 1 (derecha), entonces la probabilidad objetivo será 0.

- Luego calculamos la pérdida utilizando la función de pérdida dada, y usamos la cinta para calcular el gradiente de la pérdida con respecto a las variables entrenables del modelo. Una vez más, estos gradientes se ajustarán más tarde, antes de aplicarlos, dependiendo de lo buena o mala que resulte ser la acción.

* Finalmente, jugamos la acción seleccionada y devolvemos la nueva observación, la recompensa, si el episodio ha terminado o no, si está truncado o no y, por supuesto, los gradientes que acabamos de calcular.

Ahora creemos otra función que se basará en la función `play_one_step()` para reproducir múltiples episodios, devolviendo todas las recompensas y gradientes para cada episodio y cada paso:

In [ ]:
def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        obs, info = env.reset()
        for step in range(n_max_steps):
            obs, reward, done, truncated, grads = play_one_step(
                env, obs, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done or truncated:
                break

        all_rewards.append(current_rewards)
        all_grads.append(current_grads)

    return all_rewards, all_grads

Este código devuelve una lista de listas de recompensas: una lista de recompensas por episodio, que contiene una recompensa por paso. También devuelve una lista de listas de gradientes: una lista de gradientes por episodio, cada una de las cuales contiene una tupla de gradientes por paso y cada tupla contiene un tensor de gradiente por variable entrenable.

El algoritmo utilizará la función `play_multiple_episodes()` para jugar varias veces (por ejemplo, 10 veces), luego regresará y observará todas las recompensas, las descontará y las normalizará. Para hacer eso, necesitamos un par de funciones más; el primero calculará la suma de futuras recompensas con descuento en cada paso, y el segundo normalizará todas estas recompensas con descuento (es decir, los rendimientos) en muchos episodios restando la media y dividiéndola por la desviación estándar:

In [ ]:
def discount_rewards(rewards, discount_factor):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_factor
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_factor):
    all_discounted_rewards = [discount_rewards(rewards, discount_factor)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std
            for discounted_rewards in all_discounted_rewards]

Vamos a comprobar que esto funciona:

In [ ]:
discount_rewards([10, 0, -50], discount_factor=0.8)
#array([-22, -40, -50])

discount_and_normalize_rewards([[10, 0, -50], [10, 20]], discount_factor=0.8)
#...
#[array([-0.28435071, -0.86597718, -1.18910299]),
# array([1.26665318, 1.0727777 ])]

La llamada a `discount_rewards()` devuelve exactamente lo que esperamos (ver Figura 18-6). Puedes verificar que la función `discount_and_normalize_rewards()` efectivamente devuelve las ventajas de acción normalizadas para cada acción en ambos episodios. Observe que el primer episodio fue mucho peor que el segundo, por lo que sus ventajas normalizadas son todas negativas; todas las acciones del primer episodio se considerarían malas y, a la inversa, todas las acciones del segundo episodio se considerarían buenas.

¡Estamos casi listos para ejecutar el algoritmo! Ahora definamos los hiperparámetros. Ejecutaremos 150 iteraciones de entrenamiento, reproduciendo 10 episodios por iteración, y cada episodio durará como máximo 200 pasos. Utilizaremos un factor de descuento de 0,95:

In [ ]:
n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_factor = 0.95

También necesitamos un optimizador y la función de pérdida. Un optimizador Nadal regular con una tasa de aprendizaje de 0,01 funcionará bien, y usaremos la función de pérdida de entropía cruzada binaria porque estamos entrenando a un clasificador binario (hay dos acciones posibles: izquierda o derecha):

In [ ]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.binary_crossentropy

¡Ahora estamos listos para construir y ejecutar el bucle de entrenamiento!

In [ ]:
for iteration in range(n_iterations):
    all_rewards, all_grads = play_multiple_episodes(
        env, n_episodes_per_update, n_max_steps, model, loss_fn)
    all_final_rewards = discount_and_normalize_rewards(all_rewards,
                                                       discount_factor)
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index]
             for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis=0)
        all_mean_grads.append(mean_grads)

    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))

Vamos a revisar este código:

* En cada iteración de entrenamiento, este bucle llama a la función `play_multiple_episodes()`, que reproduce 10 episodios y devuelve las recompensas y los gradientes de cada paso de cada episodio.

- Then we call the `discount_and_normalize_rewards()` function to compute each action’s normalized advantage, called the `final_reward` in this code. This provides a measure of how good or bad each action actually was, in hindsight.

* A continuación, revisamos cada variable entrenable, y para cada una de ellas calculamos la media ponderada de los gradientes para esa variable en todos los episodios y todos los pasos, ponderada por el `final_reward`.

- Finalmente, aplicamos estos gradientes medios utilizando el optimizador: las variables entrenables del modelo se ajustarán, y esperamos que la política sea un poco mejor.

¡Y hemos terminado! Este código entrenará la política de la red neuronal y aprenderá con éxito a equilibrar el polo en el carrito. La recompensa media por episodio se acercará a los 200. De forma predeterminada, ese es el máximo para este entorno. ¡Éxito!

El algoritmo simple de gradientes de políticas que acabamos de entrenar resolvió la tarea de CartPole, pero no se escalaría bien a tareas más grandes y complejas. De hecho, es muy ineficiente, lo que significa que necesita explorar el juego durante mucho tiempo antes de que pueda hacer un progreso significativo. Esto se debe al hecho de que debe ejecutar varios episodios para estimar la ventaja de cada acción, como hemos visto. Sin embargo, es la base de algoritmos más potentes, como los algoritmos actor-crítico (que discutiremos brevemente al final de este capítulo).

#### TIP

Los investigadores tratan de encontrar algoritmos que funcionen bien incluso cuando el agente inicialmente no sabe nada sobre el medio ambiente. Sin embargo, a menos que esté escribiendo un artículo, no debe dudar en inyectar conocimientos previos en el agente, ya que acelerará drásticamente el entrenamiento. Por ejemplo, ya que sabes que el poste debe ser lo más vertical posible, podrías añadir recompensas negativas proporcionales al ángulo del poste. Esto hará que las recompensas sean mucho menos escasas y acelerará el entrenamiento. Además, si ya tiene una política razonablemente buena (por ejemplo, codificada en el redo), es posible que desee entrenar a la red neuronal para que la imite antes de usar gradientes de política para mejorarla.

#### --------------------------------------------------------------------------------------------------------------

Ahora veremos otra familia popular de algoritmos. Mientras que los algoritmos de PG intentan optimizar directamente la política para aumentar las recompensas, los algoritmos que exploraremos ahora son menos directos: el agente aprende a estimar el rendimiento esperado para cada estado, o para cada acción en cada estado, y luego utiliza este conocimiento para decidir cómo actuar. Para entender estos algoritmos, primero debemos considerar los procesos de decisión de Markov (MDP).


# Procesos de decisión de Markov

A principios del siglo XX, el matemático Andrey Markov estudió procesos estocásticos sin memoria, llamados cadenas de Markov. Tal proceso tiene un número fijo de estados, y evoluciona aleatoriamente de un estado a otro en cada paso. La probabilidad de que evolucione de un estado s a un estado s′ es fija, y depende solo del par (s, s′), no de los estados pasados. Por eso decimos que el sistema no tiene memoria.

La figura 18-7 muestra un ejemplo de una cadena de Markov con cuatro estados.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1807.png)

(_Figura 18-7. Ejemplo de una cadena de Markov_)

Supongamos que el proceso comienza en el estado s0, y hay un 70% de probabilidades de que permanezca en ese estado en el siguiente paso. Eventualmente está obligado a abandonar ese estado y nunca volver, porque ningún otro estado apunta de vuelta a s0. Si va al estado s1, lo más probable es que vaya al estado s2 (90% de probabilidad), y luego volverá inmediatamente al estado s1 (con un 100 % de probabilidad). Puede alternar varias veces entre estos dos estados, pero eventualmente caerá en el estado s3 y permanecerá allí para siempre, ya que no hay salida: esto se llama estado terminal. Las cadenas de Markov pueden tener una dinámica muy diferente, y se utilizan en gran medida en termodinámica, química, estadística y mucho más.

Los procesos de decisión de Markov fueron descritos por primera vez en la década de 1950 por Richard Bellman.⁠ Se parecen a las cadenas de Markov, pero con un giro: en cada paso, un agente puede elegir una de varias acciones posibles, y las probabilidades de transición dependen de la acción elegida. Además, algunas transiciones estatales devuelven alguna recompensa (positiva o negativa), y el objetivo del agente es encontrar una política que maximice la recompensa con el tiempo.

Por ejemplo, el MDP representado en la Figura 18-8 tiene tres estados (representados por círculos) y hasta tres posibles acciones discretas en cada paso (representadas por diamantes).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1808.png)

(_Figura 18-8. Ejemplo de un proceso de decisión de Markov_)

Si comienza en el estado s0, el agente puede elegir entre las acciones a0, a1 o a2. Si elige la acción a1, solo permanece en el estado s0 con certeza y sin ninguna recompensa. Por lo tanto, puede decidir quedarse allí para siempre si quiere. Pero si elige la acción a0, tiene una probabilidad del 70% de ganar una recompensa de +10 y permanecer en el estado s0. Luego puede intentar una y otra vez para obtener la mayor cantidad de recompensa posible, pero en un momento dado va a terminar en el estado s1. En el estado s1 solo tiene dos acciones posibles: a0 o a2. Puede optar por quedarse en su lugar eligiendo repetidamente la acción a0, o puede optar por pasar al estado s2 y obtener una recompensa negativa de -50 (ouch). En el estado s2 no tiene más remedio que tomar medidas a1, lo que muy probablemente lo llevará de vuelta al estado s0, ganando una recompensa de +40 en el camino. Tú consigues la imagen. Al mirar este MDP, ¿puedes adivinar qué estrategia obtendrá la mayor recompensa con el tiempo? En el estado s0 está claro que la acción a0 es la mejor opción, y en el estado s2 el agente no tiene más remedio que tomar la accióna1, pero en el estado s1 no es obvio si el agente debe permanecer quieto (a0) o pasar por el fuego (a2).

Bellman encontró una manera de estimar el valor de estado óptimo de cualquier estado, señaló V*(s), que es la suma de todas las recompensas futuras descontadas que el agente puede esperar en promedio después de llegar al estado, suponiendo que actúe de manera óptima. Demostró que si el agente actúa de manera óptima, entonces se aplica la ecuación de optimización de Bellman (ver Ecuación 18-1). Esta ecuación recursiva dice que si el agente actúa de manera óptima, entonces el valor óptimo del estado actual es igual a la recompensa que obtendrá en promedio después de tomar una acción óptima, más el valor óptimo esperado de todos los estados posteriores posibles a los que puede conducir esta acción.

### Ecuación 18-1. Ecuación de optimización de Bellman

![](<a href="https://imgbb.com/"><img src="https://i.ibb.co/51fh8Bn/Captura-de-pantalla-2024-03-30-a-las-19-39-25.png" alt="Captura-de-pantalla-2024-03-30-a-las-19-39-25" border="0"></a><br /><a target='_blank' href='https://imgbb.com/'>how to make a picture higher resolution</a><br />)

En esta ecuación:

* **T(s, a, s′)** es la probabilidad de transición del estado s al estado s′, dado que el agente eligió la acción a. Por ejemplo, en la Figura 18-8, **T(s2, a1, s0) = 0,8**.

* **R(s, a, s′)** es la recompensa que el agente recibe cuando pasa del estado s al estado s′, dado que el agente eligió la acción a. Por ejemplo, en la Figura 18-8, **R(s2, a1,s0) = +40**.

- **γ** es el factor de descuento.

Esta ecuación conduce directamente a un algoritmo que puede estimar con precisión el valor de estado óptimo de cada estado posible: primero inicialice todas las estimaciones de valor de estado a cero, y luego actualícelas de forma iterativa utilizando el algoritmo de iteración de valor (ver Ecuación 18-2). Un resultado notable es que, dado el tiempo suficiente, se garantiza que estas estimaciones convergan a los valores estatales óptimos, correspondientes a la política óptima.


### Ecuación 18-2. Algoritmo de iteración de valor

<a href="https://imgbb.com/"><img src="https://i.ibb.co/hL79m9R/Captura-de-pantalla-2024-03-30-a-las-19-41-24.png" alt="Captura-de-pantalla-2024-03-30-a-las-19-41-24" border="0"></a>

En esta ecuación, Vk(s) es el valor estimado del estado s en la k-iteración del algoritmo.

#### NOTA

Este algoritmo es un ejemplo de programación dinámica, que descompone un problema complejo en subproblemas manejables que se pueden abordar de forma iterativa.

#### --------------------------------------------------------------------------------------------------------------

Conocer los valores de estado óptimos puede ser útil, en particular para evaluar una política, pero no nos da la política óptima para el agente. Afortunadamente, Bellman encontró un algoritmo muy similar para estimar los valores óptimos de acción del estado, generalmente llamados valores Q (valores de calidad). El valor Q óptimo del par estado-acción (s, a), notedQ*(s, a), es la suma de las recompensas futuras descontadas que el agente puede esperar en promedio después de llegar al estado y elegir la acción a, pero antes de ver el resultado de esta acción, asumiendo que actúa de manera óptima después de esa acción.

Echemos un vistazo a cómo funciona. Una vez más, comienza inicializando todas las estimaciones de valor Q a cero, luego las actualiza utilizando el algoritmo de iteración de valor Q (consulte la ecuación 18-3).

### Ecuación 18-3. Algoritmo de iteración de valor Q

<a href="https://imgbb.com/"><img src="https://i.ibb.co/jyB51Qk/Captura-de-pantalla-2024-03-30-a-las-19-43-02.png" alt="Captura-de-pantalla-2024-03-30-a-las-19-43-02" border="0"></a>

Una vez que tenga los valores Q óptimos, definir la política óptima, señalada π*(s), es trivial; cuando el agente está en estado s, debe elegir la acción con el valor Q más alto para ese estado: <a href="https://imgbb.com/"><img src="https://i.ibb.co/Mpqc6ZQ/Captura-de-pantalla-2024-03-30-a-las-19-43-49.png" alt="Captura-de-pantalla-2024-03-30-a-las-19-43-49" border="0"></a>

Apliquemos este algoritmo al MDP representado en la Figura 18-8. En primer lugar, tenemos que definir el MDP:

In [ ]:
transition_probabilities = [  # shape=[s, a, s']
    [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
    [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
    [None, [0.8, 0.1, 0.1], None]
]
rewards = [  # shape=[s, a, s']
    [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
    [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
    [[0, 0, 0], [+40, 0, 0], [0, 0, 0]]
]
possible_actions = [[0, 1, 2], [0, 2], [1]]

Por ejemplo, para conocer la probabilidad de transición de pasar de s2 a s0 después de realizar la acción a1, buscaremos `transition_probabilities[2][1][0]` (que es 0,8). De manera similar, para obtener la recompensa correspondiente, buscaremos `rewards[2][1][0]` (que es +40). Y para obtener la lista de posibles acciones en s2, buscaremos `possible_actions[2]` (en este caso, solo es posible la acción a1). A continuación, debemos inicializar todos los valores Q a cero (excepto las acciones imposibles, para las cuales establecemos los valores Q en –∞):

In [ ]:
Q_values = np.full((3, 3), -np.inf)  # -np.inf for impossible actions
for state, actions in enumerate(possible_actions):
    Q_values[state, actions] = 0.0  # for all possible actions

Ahora ejecutemos el algoritmo de iteración del valor Q. Aplica la ecuación 18-3 repetidamente, a todos los valores Q, para cada estado y cada acción posible:

In [ ]:
gamma = 0.90  # the discount factor

for iteration in range(50):
    Q_prev = Q_values.copy()
    for s in range(3):
        for a in possible_actions[s]:
            Q_values[s, a] = np.sum([
                    transition_probabilities[s][a][sp]
                    * (rewards[s][a][sp] + gamma * Q_prev[sp].max())
                for sp in range(3)])

¡Eso es todo! Los valores Q resultantes se ven así:

In [ ]:
Q_values

#array([[18.91891892, 17.02702702, 13.62162162],
#       [ 0.        ,        -inf, -4.87971488],
#       [       -inf, 50.13365013,        -inf]])

Por ejemplo, cuando el agente está en el estado s0 y elige la acción a1, la suma esperada de recompensas futuras con descuento es de aproximadamente 17,0.

Para cada estado, podemos encontrar la acción que tiene el valor Q más alto:

In [ ]:
Q_values.argmax(axis=1)  # optimal action for each state

#array([0, 0, 1])

Esto nos da la política óptima para este MDP cuando se utiliza un factor de descuento de 0,90: en el estado s0 elija la acción a0, en el estado s1 elija la acción a0 (es decir, quédese en su lugar), y en los estados 2 elija la acción a1 (la única acción posible). Curiosamente, si aumentamos el factor de descuento a 0,95, la política óptima cambia: en el estado s1 la mejor acción se convierte en a2 (¡pasa por el fuego!). Esto tiene sentido porque cuanto más valoras las recompensas futuras, más estás dispuesto a soportar un poco de dolor ahora para la promesa de felicidad futura.


# Aprendizaje de diferencias temporales


Los problemas de aprendizaje de refuerzo con acciones discretas a menudo se pueden modelar como procesos de decisión de Markov, pero el agente inicialmente no tiene idea de cuáles son las probabilidades de transición (no conoce T(s, a, s′)), y tampoco sabe cuáles van a ser las recompensas (no conoce R(s, a, s′)). Debe experimentar cada estado y cada transición al menos una vez para conocer las recompensas, y debe experimentarlas varias veces si se va a tener una estimación razonable de las probabilidades de transición.

El algoritmo de aprendizaje de la diferencia temporal (TD) es muy similar al algoritmo de iteración del valor Q, pero se ha ajustado para tener en cuenta el hecho de que el agente solo tiene un conocimiento parcial del MDP. En general, asumimos que el agente inicialmente solo conoce los posibles estados y acciones, y nada más. El agente utiliza una política de exploración, por ejemplo, una política puramente aleatoria, para explorar el MDP, y a medida que avanza, el algoritmo de aprendizaje de TD actualiza las estimaciones de los valores de estado en función de las transiciones y recompensas que realmente se observan (ver Ecuación 18-4).


### Ecuación 18-4. Algoritmo de aprendizaje de TD

<a href="https://ibb.co/gd089y6"><img src="https://i.ibb.co/1MCNQZb/Captura-de-pantalla-2024-03-30-a-las-19-48-34.png" alt="Captura-de-pantalla-2024-03-30-a-las-19-48-34" border="0"></a>

#### TIP

El aprendizaje de TD tiene muchas similitudes con el descenso de gradiente estocástico, incluido el hecho de que maneja una muestra a la vez. Además, al igual que SGD, solo puede converger realmente si reduce gradualmente la tasa de aprendizaje; de lo contrario, seguirá rebotando alrededor de los valores Q óptimos.

#### --------------------------------------------------------------------------------------------------------------

Para cada estado, este algoritmo realiza un seguimiento de un promedio de las recompensas inmediatas que el agente obtiene al salir de ese estado, además de las recompensas que espera obtener más tarde, suponiendo que actúe de manera óptima.


# Aprendizaje de preguntas

Del mismo modo, el algoritmo de aprendizaje Q es una adaptación del algoritmo de iteración del valor Q a la situación en la que las probabilidades de transición y las recompensas son inicialmente desconocidas (ver Ecuación 18-5). Q-learning funciona viendo a un agente jugar (por ejemplo, al azar) y mejorando gradualmente sus estimaciones de los valores Q. Una vez que tiene estimaciones precisas del valor Q (o lo suficientemente cerca), la política óptima es elegir la acción que tiene el valor Q más alto (es decir, la política codiciosa).

### Ecuación 18-5. Algoritmo de aprendizaje Q

<a href="https://imgbb.com/"><img src="https://i.ibb.co/sFST4dz/Captura-de-pantalla-2024-03-30-a-las-19-49-59.png" alt="Captura-de-pantalla-2024-03-30-a-las-19-49-59" border="0"></a>

Para cada par estado-acción (s, a), este algoritmo realiza un seguimiento de un promedio de las recompensas que obtiene el agente al salir de los estados con la acción a, más la suma de las recompensas futuras con descuento que espera obtener. Para estimar esta suma, tomamos el máximo de las estimaciones del valor Q para el próximo estado s′, ya que asumimos que la política objetivo actuará de manera óptima a partir de entonces.

Implementemos el algoritmo de aprendizaje Q. En primer lugar, tendremos que hacer que un agente explore el entorno. Para esto, necesitamos una función de paso para que el agente pueda ejecutar una acción y obtener el estado y la recompensa resultantes:

In [ ]:
def step(state, action):
    probas = transition_probabilities[state][action]
    next_state = np.random.choice([0, 1, 2], p=probas)
    reward = rewards[state][action][next_state]
    return next_state, reward

Ahora implementemos la política de exploración del agente. Dado que el espacio estatal es bastante pequeño, una simple política aleatoria será suficiente. Si ejecutamos el algoritmo durante el tiempo suficiente, el agente visitará todos los estados muchas veces, y también intentará todas las acciones posibles muchas veces:

In [ ]:
def exploration_policy(state):
    return np.random.choice(possible_actions[state])

A continuación, después de inicializar los valores Q como antes, estamos listos para ejecutar el algoritmo de aprendizaje Q con decadencia de la tasa de aprendizaje (utilizando la programación de energía, introducida en el capítulo 11):

In [ ]:
alpha0 = 0.05  # initial learning rate
decay = 0.005  # learning rate decay
gamma = 0.90  # discount factor
state = 0  # initial state

for iteration in range(10_000):
    action = exploration_policy(state)
    next_state, reward = step(state, action)
    next_value = Q_values[next_state].max()  # greedy policy at the next step
    alpha = alpha0 / (1 + iteration * decay)
    Q_values[state, action] *= 1 - alpha
    Q_values[state, action] += alpha * (reward + gamma * next_value)
    state = next_state

Este algoritmo convergerá a los valores Q óptimos, pero tomará muchas iteraciones y, posiblemente, bastante ajuste de hiperparámetros. Como se puede ver en la Figura 18-9, el algoritmo de iteración del valor Q (izquierda) converge muy rápidamente, en menos de 20 iteraciones, mientras que el algoritmo de aprendizaje Q (derecha) tarda alrededor de 8.000 iteraciones en converger. ¡Obviamente, no conocer las probabilidades de transición o las recompensas hace que encontrar la política óptima sea significativamente más difícil!

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1809.png)

![](Figura 18-9. Curva de aprendizaje del algoritmo de iteración del valor Q frente al algoritmo de aprendizaje Q)

El algoritmo Q-learning se llama algoritmo fuera de la política porque la política que se está entrenando no es necesariamente la que se utiliza durante el entrenamiento. Por ejemplo, en el código que acabamos de ejecutar, la política que se estaba ejecutando (la política de exploración) fue completamente aleatoria, mientras que la política que se estaba entrenando nunca se utilizó. Después de la capacitación, la política óptima corresponde a la elección sistemática de la acción con el valor Q más alto. Por el contrario, el algoritmo de gradientes de políticas es un algoritmo de política: explora el mundo utilizando la política que se está entrenando. Es algo sorprendente que Q-learning sea capaz de aprender la política óptima con solo ver a un agente actuar al azar. Imagina aprender a jugar al golf cuando tu profesor es un mono con los ojos vendados. ¿Podemos hacerlo mejor?


## Políticas de exploración

Por supuesto, Q-learning solo puede funcionar si la política de exploración explora el MDP a fondo. Aunque se garantiza que una política puramente aleatoria finalmente visitará todos los estados y cada transición muchas veces, puede llevar mucho tiempo hacerlo. Por lo tanto, una mejor opción es usar la política ε-codiciosa (ε es epsilon): en cada paso actúa al azar con la probabilidad ε, o con avaricia con la probabilidad 1-ε (es decir, elegir la acción con el valor Q más alto). La ventaja de la política codiciosa (en comparación con una política completamente aleatoria) es que pasará cada vez más tiempo explorando las partes interesantes del medio ambiente, a medida que las estimaciones del valor Q mejoran cada vez más, mientras que todavía pasa algún tiempo visitando regiones desconocidas del MDP. Es bastante común comenzar con un valor alto para ε (por ejemplo, 1,0) y luego reducirlo gradualmente (por ejemplo, hasta 0,05).

Alternativamente, en lugar de confiar solo en la oportunidad para la exploración, otro enfoque es alentar a la política de exploración a probar acciones que no ha intentado mucho antes. Esto se puede implementar como un bono añadido a las estimaciones del valor Q, como se muestra en la Ecuación 18-6.

### Ecuación 18-6. Q-learning usando una función de exploración

![](<a href="https://imgbb.com/"><img src="https://i.ibb.co/HNPV5g4/Captura-de-pantalla-2024-03-30-a-las-19-53-08.png" alt="Captura-de-pantalla-2024-03-30-a-las-19-53-08" border="0"></a>)

En esta ecuación:

- **N(s′, a′)** cuenta el número de veces que se eligió la acción a′ en el estado s′.

* **f(Q, N)** es una función de exploración, como f(Q, N) = Q + κ/(1 + N), donde κ es un hiperparámetro de curiosidad que mide cuánto se siente atraído el agente por lo desconocido.


## Aprendizaje Q aproximado y aprendizaje Q profundo

El principal problema con Q-learning es que no escala bien a MDP grandes (o incluso medianos) con muchos estados y acciones. Por ejemplo, supongamos que querías usar Q-learning para entrenar a un agente para interpretar a la Sra. Pac-Man (ver Figura 18-1). Hay alrededor de 150 pellets que la Sra. Pac-Man puede comer, cada uno de los cuales puede estar presente o ausente (es decir, ya comido). Por lo tanto, el número de estados posibles es mayor que 2150 ≈ 1045. Y si agrega todas las combinaciones posibles de posiciones para todos los fantasmas y la Sra. Pac-Man, el número de estados posibles se vuelve mayor que el número de átomos en nuestro planeta, por lo que no hay absolutamente ninguna manera de que pueda hacer un seguimiento de una estimación para cada valor Q.

La solución es encontrar una función Qθ(s, a) que se aproxime al valor Q de cualquier par estado-acción (s, a) utilizando un número manejable de parámetros (dado por el vector de parámetros θ). Esto se llama aprendizaje Q-aprendizaje aproximado. Durante años se recomendó utilizar combinaciones lineales de características hechas a mano extraídas del estado (por ejemplo, las distancias de los fantasmas más cercanos, sus direcciones, etc.) para estimar los valores Q, pero en 2013, DeepMind demostró que el uso de redes neuronales profundas puede funcionar mucho mejor, especialmente para problemas complejos, y no requiere ninguna ingeniería de características. Un DNN utilizado para estimar los valores Q se llama red Q profunda (DQN), y el uso de un DQN para el aprendizaje Q aproximado se llama aprendizaje Q profundo.

Ahora, ¿cómo podemos entrenar a un DQN? Bueno, considere el valor Q aproximado calculado por el DQN para un par (s) de estado-acción (s, a). Gracias a Bellman, sabemos que queremos que este valor Q aproximado esté lo más cerca posible de la recompensa r que realmente observamos después de jugar la acción a en el estado s, más el valor descontado de jugar de manera óptima a partir de entonces. Para estimar esta suma de futuras recompensas con descuento, solo podemos ejecutar el DQN en el siguiente estado s′, para todas las acciones posibles a′. Obtenemos un valor Q futuro aproximado para cada acción posible. Luego elegimos el más alto (ya que asumimos que jugaremos de manera óptima) y lo descontamos, y esto nos da una estimación de la suma de futuras recompensas con descuento. Al sumar la recompensa r y la estimación del valor descontado futuro, obtenemos un valor Q objetivo y(s, a) para el par estado-acción (s, a), como se muestra en la ecuación 18-7.

### Ecuación 18-7. Valor Q objetivo

<a href="https://imgbb.com/"><img src="https://i.ibb.co/XzBkQpg/Captura-de-pantalla-2024-03-30-a-las-20-00-16.png" alt="Captura-de-pantalla-2024-03-30-a-las-20-00-16" border="0"></a>

Con este valor Q objetivo, podemos ejecutar un paso de entrenamiento utilizando cualquier algoritmo de descenso de gradiente. Específicamente, generalmente tratamos de minimizar el error cuadrateado entre el valor Q estimado Qθ(s, a) y el valor Q objetivo y(s, a), o la pérdida de Huber para reducir la sensibilidad del algoritmo a los errores grandes. ¡Y ese es el algoritmo de aprendizaje Q profundo! Veamos cómo implementarlo para resolver el entorno CartPole.


# Implementación de Q-Learning profundo

Lo primero que necesitamos es una red Q profunda. En teoría, necesitamos una red neuronal que tome un par estado-acción como entrada y genere un valor Q aproximado. Sin embargo, en la práctica es mucho más eficiente usar una red neuronal que toma solo un estado como entrada y genera un valor Q aproximado para cada acción posible. Para resolver el entorno CartPole, no necesitamos una red neuronal muy complicada; un par de capas ocultas harán:

In [ ]:
input_shape = [4]  # == env.observation_space.shape
n_outputs = 2  # == env.action_space.n

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="elu", input_shape=input_shape),
    tf.keras.layers.Dense(32, activation="elu"),
    tf.keras.layers.Dense(n_outputs)
])

Para seleccionar una acción usando este DQN, elegemos la acción con el mayor valor Q predicho. Para asegurarnos de que el agente explore el entorno, utilizaremos una política de codicia (es decir, elegiremos una acción aleatoria con probabilidad ε):

In [ ]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model.predict(state[np.newaxis], verbose=0)[0]
        return Q_values.argmax()  # optimal action according to the DQN

En lugar de entrenar al DQN basándose solo en las últimas experiencias, almacenaremos todas las experiencias en un búfer de reproducción (o memoria de repetición), y probaremos un lote de entrenamiento aleatorio en cada iteración de entrenamiento. Esto ayuda a reducir las correlaciones entre las experiencias en un lote de entrenamiento, lo que ayuda enormemente al entrenamiento. Para esto, solo usaremos una cola de doble finte (`deque`):

In [ ]:
from collections import deque

replay_buffer = deque(maxlen=2000)

#### TIP

Una deque es una cola de elementos que se pueden agregar o eliminar de manera eficiente en ambos extremos. Insertar y eliminar elementos de los extremos de la cola es muy rápido, pero el acceso aleatorio puede ser lento cuando la cola se alarga. Si necesita un búfer de reproducción muy grande, debe usar un búfer circular en su lugar (consulte el cuaderno para ver una implementación), o consulte la biblioteca Reverb de DeepMind.

#### ---------------------------------------------------------------------------------------------------------------

Cada experiencia estará compuesta por seis elementos: un estado s, la acción a que el agente tomó, la recompensa resultante r, el siguiente estado s′ que alcanzó, un booleano que indique si el episodio terminó en ese punto (done) y finalmente otro booleano que indique si el episodio fue truncado en ese momento. Necesitaremos una pequeña función para muestrear un lote aleatorio de experiencias del búfer de reproducción. Devolverá seis matrices NumPy correspondientes a los seis elementos de la experiencia:

In [ ]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    return [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(6)
    ]  # [states, actions, rewards, next_states, dones, truncateds]

También creemos una función que jugará un solo paso usando la política ε-gredy, y luego almacenar la experiencia resultante en el búfer de reproducción:

In [ ]:
def play_one_step(env, state, epsilon):
    action = epsilon_greedy_policy(state, epsilon)
    next_state, reward, done, truncated, info = env.step(action)
    replay_buffer.append((state, action, reward, next_state, done, truncated))
    return next_state, reward, done, truncated, info

Por último, vamos a crear una última función que muestreará un lote de experiencias del búfer de reproducción y entrenará el DQN realizando un solo paso de descenso de gradiente en este lote:

In [ ]:
batch_size = 32
discount_factor = 0.95
optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-2)
loss_fn = tf.keras.losses.mean_squared_error

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones, truncateds = experiences
    next_Q_values = model.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + runs * discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

Esto es lo que está sucediendo en este código:

- Primero definimos algunos hiperparámetros y creamos el optimizador y la función de pérdida.

* Luego creamos la función `training_step()`. Comienza muestreando un lote de experiencias, luego utiliza el DQN para predecir el valor Q para cada acción posible en el siguiente estado de cada experiencia. Dado que asumimos que el agente jugará de manera óptima, solo mantenemos el valor máximo de Q para cada estado siguiente. A continuación, utilizamos la ecuación 18-7 para calcular el valor Q objetivo para el par estado-acción de cada experiencia.

- Queremos usar el DQN para calcular el valor Q para cada par de estado-acción experimentado, pero el DQN también generará los valores Q para las otras acciones posibles, no solo para la acción que realmente eligió el agente. Por lo tanto, debemos enmascarar todos los valores Q que no necesitamos. La función `tf.one_hot()` permite convertir una serie de índices de acción en dicha máscara. Por ejemplo, si las tres primeras experiencias contienen las acciones 1, 1, 0, respectivamente, entonces la máscara comenzará con `[[0, 1], [0, 1], [1, 0],...]`. Luego podemos multiplicar la salida del DQN con esta máscara, y esto pondrá a cero todos los valores Q que no queremos. Luego sumamos sobre el eje 1 para eliminar todos los ceros, manteniendo solo los valores Q de los pares estado-acción experimentados. Esto nos da el tensor `Q_values`, que contiene un valor Q predicho para cada experiencia del lote.

* A continuación, calculamos la pérdida: es el error cuadrático medio entre los valores Q objetivo y los predichos para los pares estado-acción experimentados.

- Finalmente, realizamos un paso de descenso de gradiente para minimizar la pérdida con respecto a las variables entrenables del modelo.

Esta fue la parte más difícil. Ahora entrenar al modelo es sencillo:

In [ ]:
for episode in range(600):
    obs, info = env.reset()
    for step in range(200):
        epsilon = max(1 - episode / 500, 0.01)
        obs, reward, done, truncated, info = play_one_step(env, obs, epsilon)
        if done or truncated:
            break

    if episode > 50:
        training_step(batch_size)

Ejecutamos 600 episodios, cada uno con un máximo de 200 pasos. En cada paso, primero calculamos el valor épsilon para la política ε-codiciosa: irá de 1 a 0,01, linealmente, en poco menos de 500 episodios. Luego llamamos a la función `play_one_step()`, que utilizará la política ε-greedy para elegir una acción, luego ejecutarla y registrar la experiencia en el búfer de reproducción. Si el episodio finaliza o se trunca, salimos del bucle. Finalmente, si ya pasamos el episodio 50, llamamos a la función `training_step()` para entrenar el modelo en un lote muestreado del búfer de reproducción. La razón por la que reproducimos muchos episodios sin entrenamiento es para darle tiempo al búfer de repetición para que se llene (si no esperamos lo suficiente, entonces no habrá suficiente diversidad en el búfer de reproducción). Y eso es todo: ¡acabamos de implementar el algoritmo Deep Q-learning!

La figura 18-10 muestra las recompensas totales que el agente recibió durante cada episodio.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1810.png)

(_Figura 18-10. Curva de aprendizaje del algoritmo de aprendizaje Q profundo_)

Como puede ver, el algoritmo tardó un poco en empezar a aprender algo, en parte porque ε era muy alto al principio. Luego, su progreso fue errático: primero alcanzó la recompensa máxima alrededor del episodio 220, pero inmediatamente cayó, luego rebotó hacia arriba y hacia abajo unas cuantas veces, y poco después parecía que finalmente se había estabilizado cerca de la recompensa máxima, alrededor del episodio 320. su puntuación volvió a caer drásticamente. A esto se le llama olvido catastrófico y es uno de los grandes problemas que enfrentan prácticamente todos los algoritmos de RL: a medida que el agente explora el entorno, actualiza su política, pero lo que aprende en una parte del entorno puede alterar lo que aprendió anteriormente en otras. partes del medio ambiente. Las experiencias están bastante correlacionadas y el entorno de aprendizaje sigue cambiando; ¡esto no es ideal para el descenso de gradientes! Si aumenta el tamaño del búfer de reproducción, el algoritmo estará menos sujeto a este problema. Ajustar la tasa de aprendizaje también puede ayudar. Pero la verdad es que el aprendizaje por refuerzo es difícil: el entrenamiento suele ser inestable y es posible que tengas que probar muchos valores de hiperparámetros y semillas aleatorias antes de encontrar una combinación que funcione bien. Por ejemplo, si intenta cambiar la función de activación de `"elu"` a `"relu"`, el rendimiento será mucho menor.

#### NOTA

El aprendizaje de refuerzo es notoriamente difícil, en gran parte debido a las inestabilidades del entrenamiento y la enorme sensibilidad a la elección de los valores de hiperparámetros y las semillas aleatorias.⁠13 Como dijo el investigador Andrej Karpathy, "[El aprendizaje supervisado] quiere funcionar. [... ] RL debe ser obligado a trabajar". Necesitarás tiempo, paciencia, perseverancia y tal vez un poco de suerte también. Esta es una de las principales razones por las que RL no es tan ampliamente adoptado como el aprendizaje profundo regular (por ejemplo, redes convolucionales). Pero hay algunas aplicaciones del mundo real, más allá de los juegos de AlphaGo y Atari: por ejemplo, Google utiliza RL para optimizar los costos de su centro de datos, y se utiliza en algunas aplicaciones de robótica, para el ajuste de hiperparámetros y en sistemas de recomendación.

#### --------------------------------------------------------------------------------------------------------------

Puede que te preguntes por qué no planeamos la pérdida. Resulta que la pérdida es un indicador deficiente del rendimiento del modelo. La pérdida podría bajar, pero el agente podría tener un rendimiento peor (por ejemplo, esto puede suceder cuando el agente se queda atascado en una pequeña región del entorno, y el DQN comienza a sobreacostar esta región). Por el contrario, la pérdida podría aumentar, sin embargo, el agente podría funcionar mejor (por ejemplo, si el DQN estaba subestimando los valores Q y comienza a aumentar correctamente sus predicciones, es probable que el agente tenga un mejor rendimiento, obteniendo más recompensas, pero la pérdida podría aumentar porque el DQN también establece los objetivos, que también serán más grandes). Por lo tanto, es preferible trazar las recompensas.

El algoritmo básico de aprendizaje Q profundo que hemos estado usando hasta ahora sería demasiado inestable para aprender a jugar a los juegos de Atari. Entonces, ¿cómo lo hizo DeepMind? Bueno, ¡modificaron el algoritmo!


# Variantes profundas de Q-Learning


Echemos un vistazo a algunas variantes del algoritmo de aprendizaje Q profundo que puede estabilizar y acelerar el entrenamiento.

## Objetivos de valor Q fijos

En el algoritmo básico de aprendizaje Q profundo, el modelo se utiliza tanto para hacer predicciones como para establecer sus propios objetivos. Esto puede llevar a una situación análoga a la de un perro que persigue su propia cola. Este bucle de retroalimentación puede hacer que la red sea inestable: puede divergir, oscilar, congelar, etc. Para resolver este problema, en su documento de 2013, los investigadores de DeepMind utilizaron dos DQN en lugar de uno: el primero es el modelo en línea, que aprende en cada paso y se utiliza para mover al agente, y el otro es el modelo objetivo utilizado solo para definir los objetivos. El modelo de destino es solo un clon del modelo en línea:

In [ ]:
target = tf.keras.models.clone_model(model)  # clone the model's architecture
target.set_weights(model.get_weights())  # copy the weights

Luego, en la función `training_step()`, solo necesitamos cambiar una línea para usar el modelo de destino en lugar del modelo en línea al calcular los valores Q de los siguientes estados:

In [ ]:
next_Q_values = target.predict(next_states, verbose=0)

Por último, en el bucle de entrenamiento, debemos copiar los pesos del modelo en línea al modelo objetivo, a intervalos regulares (por ejemplo, cada 50 episodios):

In [ ]:
if episode % 50 == 0:
    target.set_weights(model.get_weights())

Dado que el modelo objetivo se actualiza con mucha menos frecuencia que el modelo en línea, los objetivos de valor Q son más estables, el ciclo de retroalimentación que analizamos anteriormente se amortigua y sus efectos son menos severos. Este enfoque fue una de las principales contribuciones de los investigadores de DeepMind en su artículo de 2013, permitiendo a los agentes aprender a jugar juegos de Atari a partir de píxeles sin procesar. Para estabilizar el entrenamiento, utilizaron una pequeña tasa de aprendizaje de 0,00025, actualizaron el modelo objetivo solo cada 10.000 pasos (en lugar de 50) y utilizaron un búfer de reproducción muy grande de 1 millón de experiencias. Disminuyeron `epsilon` muy lentamente, de 1 a 0,1 en 1 millón de pasos, y dejaron que el algoritmo se ejecutara durante 50 millones de pasos. Además, su DQN era una red convolucional profunda.

Ahora echamos un vistazo a otra variante de DQN que logró superar el estado del arte una vez más.

## Doble DQN

En un artículo de 2015, investigadores de DeepMind ajustaron su algoritmo DQN, aumentando su rendimiento y estabilizando un poco el entrenamiento. Llamaron a esta variante doble DQN. La actualización se basó en la observación de que la red de destino es propensa a sobreestimar los valores Q. De hecho, supongamos que todas las acciones son igualmente buenas: los valores Q estimados por el modelo objetivo deben ser idénticos, pero como son aproximaciones, algunas pueden ser ligeramente mayores que otras, por pura casualidad. El modelo objetivo siempre seleccionará el valor Q más grande, que será ligeramente mayor que el valor Q medio, lo más probable es que sobreestime el valor Q real (un poco como contar la altura de la onda aleatoria más alta al medir la profundidad de una piscina). Para solucionar esto, los investigadores propusieron usar el modelo en línea en lugar del modelo de destino al seleccionar las mejores acciones para los próximos estados, y usar el modelo objetivo solo para estimar los valores Q para estas mejores acciones. Aquí está la función updatedtraining `training_step()`:

In [ ]:
def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones, truncateds = experiences
    next_Q_values = model.predict(next_states, verbose=0)  # ≠ target.predict()
    best_next_actions = next_Q_values.argmax(axis=1)
    next_mask = tf.one_hot(best_next_actions, n_outputs).numpy()
    max_next_Q_values = (target.predict(next_states, verbose=0) * next_mask
                        ).sum(axis=1)
    [...]  # the rest is the same as earlier

Solo unos meses después, se propuso otra mejora en el algoritmo DQN; lo veremos a continuación.


## Repetición de experiencia prioritaria


En lugar de muestrear experiencias de manera uniforme desde el búfer de repetición, ¿por qué no muestrear experiencias importantes con más frecuencia? Esta idea se llama muestreo de importancia (IS) o reproducción de la experiencia priorizada (PER), y fue presentada en un artículo de 2015⁠ por investigadores de DeepMind (¡una vez más!).

Más específicamente, las experiencias se consideran "importantes" si es probable que conduzcan a un rápido progreso del aprendizaje. Pero, ¿cómo podemos estimar esto? Un enfoque razonable es medir la magnitud del error TD δ = r + γ·V(s′) - V(s). Un gran error de TD indica que una transición (s, a, s′) es muy sorprendente y, por lo tanto, probablemente valga la pena aprender de ella.⁠16 Cuando una experiencia se registra en el búfer de reproducción, su prioridad se establece en un valor muy grande, para garantizar que se muestree al menos una vez. Sin embargo, una vez que se muestrea (y cada vez que se muestrea), se calcula el error TD δ, y la prioridad de esta experiencia se establece en p = |δ| (además de una pequeña constante para garantizar que cada experiencia tenga una probabilidad distinta de ser muestreada). La probabilidad P de muestrear una experiencia con prioridad p es proporcional a pζ, donde ζ es un hiperparámetro que controla lo codiciosos que queremos que sea el muestreo de importancia: cuando ζ = 0, solo obtenemos un muestreo uniforme, y cuando ζ = 1, obtenemos un muestreo de importancia completo. En el artículo, los autores usaron ζ = 0,6, pero el valor óptimo dependerá de la tarea.

Sin embargo, hay un trago: dado que las muestras estarán sesgadas hacia experiencias importantes, debemos compensar este sesgo durante el entrenamiento rebajas las experiencias de acuerdo con su importancia, o de lo contrario el modelo simplemente se sobreajustará a las experiencias importantes. Para que quede claro, queremos que las experiencias importantes se muestreen con más frecuencia, pero esto también significa que debemos darles un peso más bajo durante el entrenamiento. Para hacer esto, definimos el peso de entrenamiento de cada experiencia como w = (n P)-β, donde n es el número de experiencias en el búfer de reproducción, y β es un hiperparámetro que controla cuánto queremos compensar el sesgo de muestreo de importancia (0 significa en absoluto, mientras que 1 significa completamente). En el artículo, los autores usaron β = 0,4 al comienzo del entrenamiento y lo aumentaron linealmente a β = 1 al final del entrenamiento. Una vez más, el valor óptimo dependerá de la tarea, pero si aumentas una, por lo general también querrás aumentar la otra.

Ahora echemos un vistazo a una última variante importante del algoritmo DQN.


## Duelo DQN

El algoritmo de duelo DQN (DDQN, que no debe confundirse con el doble DQN, aunque ambas técnicas se pueden combinar fácilmente) fue introducido en otro documento de 2015⁠ por los investigadores de DeepMind. Para entender cómo funciona, primero debemos tener en cuenta que el valor Q de un par de estado-acción (s, a) se puede expresar como Q(s, a) = V(s) + A(s,a), donde V(s) es el valor del estado s y A(s, a) es la ventaja de tomar la acción a en el estado s, en comparación con todas las demás acciones posibles en ese estado. Además, el valor de un estado es igual al valor Q de la mejor acción a* para ese estado (ya que asumimos que la política óptima elegirá la mejor acción), por lo que V(s) = Q(s, a*), lo que implica que A(s, a*) = 0. En un DQN de duelo, el modelo estima tanto el valor del estado como la ventaja de cada acción posible. Dado que la mejor acción debería tener una ventaja de 0, el modelo resta la ventaja máxima prevista de todas las ventajas previstas. Aquí hay un modelo DDQN simple, implementado utilizando la API funcional:

In [ ]:
input_states = tf.keras.layers.Input(shape=[4])
hidden1 = tf.keras.layers.Dense(32, activation="elu")(input_states)
hidden2 = tf.keras.layers.Dense(32, activation="elu")(hidden1)
state_values = tf.keras.layers.Dense(1)(hidden2)
raw_advantages = tf.keras.layers.Dense(n_outputs)(hidden2)
advantages = raw_advantages - tf.reduce_max(raw_advantages, axis=1,
                                            keepdims=True)
Q_values = state_values + advantages
model = tf.keras.Model(inputs=[input_states], outputs=[Q_values])

El resto del algoritmo es el mismo que antes. De hecho, ¡puedes construir un DQN de doble duelo y combinarlo con la repetición de la experiencia priorizada! De manera más general, se pueden combinar muchas técnicas de RL, como demostró DeepMind en un artículo de 2017: los autores del artículo combinaron seis técnicas diferentes en un agente llamado Rainbow, que superó en gran medida al estado del arte.

Como puedes ver, el aprendizaje por refuerzo profundo es un campo de rápido crecimiento y ¡hay mucho más por descubrir!


# Descripción general de algunos algoritmos populares de RL

Antes de cerrar este capítulo, echemos un breve vistazo a algunos otros algoritmos populares:

**AlphaGo⁠**

    AlphaGo utiliza una variante de la búsqueda de árboles de Monte Carlo (MCTS) basada en redes neuronales profundas para vencer a los campeones humanos en el juego de Go. MCTS fue inventado en 1949 por Nicholas Metropolis y Stanislaw Ulam. Selecciona el mejor movimiento después de ejecutar muchas simulaciones, explorando repetidamente el árbol de búsqueda a partir de la posición actual y pasando más tiempo en las ramas más prometedoras. Cuando llega a un nodo que no ha visitado antes, juega al azar hasta que termina el juego, y actualiza sus estimaciones para cada nodo visitado (excluyendo los movimientos aleatorios), aumentando o disminuyendo cada estimación dependiendo del resultado final. AlphaGo se basa en el mismo principio, pero utiliza una red de políticas para seleccionar movimientos, en lugar de jugar al azar. Esta red de políticas se entrena utilizando gradientes de políticas. El algoritmo original involucraba tres redes neuronales más, y era más complicado, pero se simplificó en el documento AlphaGo Zero,⁠ que utiliza una sola red neuronal tanto para seleccionar movimientos como para evaluar los estados del juego. El papel AlphaZero⁠21 generalizó este algoritmo, lo que lo hace capaz de abordar no solo el juego de Go, sino también el ajedrez y el shogi (ajedrez japonés). Por último, el papel MuZero⁠22 continuó mejorando este algoritmo, superando las iteraciones anteriores a pesar de que el agente comienza sin siquiera conocer las reglas del juego.

**Algoritmos de actor-crítico**

    Los actores-críticos son una familia de algoritmos RL que combinan gradientes de políticas con redes Q profundas. Un agente actor-crítico contiene dos redes neuronales: una red de políticas y un DQN. El DQN se entrena normalmente, aprendiendo de las experiencias del agente. La red de políticas aprende de manera diferente (y mucho más rápido) que en PG regular: en lugar de estimar el valor de cada acción pasando por múltiples episodios, luego sumar las recompensas futuras con descuento para cada acción y, finalmente, normalizarlas, el agente (actor) se basa en los valores de acción estimados por el DQN (crítico). Es un poco como un atleta (el agente) aprendiendo con la ayuda de un entrenador (el DQN).

**Ventaja asíncrona actor-crítico (A3C)⁠**

    Esta es una importante variante de actor-crítico introducida por los investigadores de DeepMind en 2016, donde múltiples agentes aprenden en paralelo, explorando diferentes copias del entorno. A intervalos regulares, pero de forma asíncrona (de ahí el nombre), cada agente envía algunas actualizaciones de peso a una red maestra, y luego extrae los últimos pesos de esa red. Por lo tanto, cada agente contribuye a mejorar la red maestra y se beneficia de lo que los otros agentes han aprendido. Además, en lugar de estimar los valores Q, el DQN estima la ventaja de cada acción (de ahí la segunda A en el nombre), que estabiliza el entrenamiento.

**Ventaja actor-crítico (A2C)**

    A2C es una variante del algoritmo A3C que elimina la asincronicidad. Todas las actualizaciones del modelo son sincrónicas, por lo que las actualizaciones de gradiente se realizan en lotes más grandes, lo que permite que el modelo utilice mejor la potencia de la GPU.

**Soft Actor-crítico (SAC)⁠**

    SAC es una variante de actor-crítico propuesta en 2018 por Tuomas Haarnoja y otros investigadores de la UC Berkeley. Aprende no solo recompensas, sino también a maximizar la entropía de sus acciones. En otras palabras, trata de ser lo más impredecible posible sin dejar de obtener tantas recompensas como sea posible. Esto alienta al agente a explorar el entorno, lo que acelera el entrenamiento, y hace que sea menos probable que ejecute repetidamente la misma acción cuando el DQN produce estimaciones imperfectas. Este algoritmo ha demostrado una increíble eficiencia de muestra (a diferencia de todos los algoritmos anteriores, que aprenden muy lentamente).

**Optimización de políticas proximales (PPO)⁠**

    Este algoritmo de John Schulman y otros investigadores de OpenAI se basa en A2C, pero corta la función de pérdida para evitar actualizaciones de peso excesivamente grandes (que a menudo conducen a inestabilidades en el entrenamiento). PPO es una simplificación del algoritmo anterior de optimización de la política de la región de confianza⁠26 (TRPO), también de OpenAI. OpenAI fue noticia en abril de 2019 con su IA llamada OpenAI Five, basada en el algoritmo de PPO, que derrotó a los campeones del mundo en el juego multijugador Dota 2.

**Exploración basada en la curiosidad⁠**

    Un problema recurrente en RL es la esparidad de las recompensas, lo que hace que el aprendizaje sea muy lento e ineficiente. Deepak Pathak y otros investigadores de UC Berkeley han propuesto una forma emocionante de abordar este problema: ¿por qué no ignorar las recompensas y simplemente hacer que el agente sea extremadamente curioso por explorar el medio ambiente? Por lo tanto, las recompensas se vuelven intrínsecas del agente, en lugar de venir del entorno. Del mismo modo, es más probable que estimular la curiosidad en un niño dé buenos resultados que simplemente recompensar al niño por obtener buenas calificaciones. ¿Cómo funciona esto? El agente intenta continuamente predecir el resultado de sus acciones, y busca situaciones en las que el resultado no coincide con sus predicciones. En otras palabras, quiere sorprenderse. Si el resultado es predecible (aburrido), va a otra parte. Sin embargo, si el resultado es impredecible, pero el agente se da cuenta de que no tiene control sobre él, también se aburre después de un tiempo. Con solo curiosidad, los autores lograron entrenar a un agente en muchos videojuegos: a pesar de que el agente no recibe penalización por perder, el juego comienza de nuevo, lo cual es aburrido, por lo que aprende a evitarlo.

**Aprendizaje abierto (OEL)**

    El objetivo de OEL es capacitar a agentes capaces de aprender sin cesar tareas nuevas e interesantes, normalmente generadas por procedimientos. Todavía no hemos llegado, pero ha habido un progreso increíble en los últimos años. Por ejemplo, un documento de 2019⁠28 de un equipo de investigadores de Uber AI introdujo el algoritmo POET, que genera múltiples entornos 2D simulados con golpes y agujeros y entrena a un agente por entorno: el objetivo del agente es caminar lo más rápido posible mientras se evita los obstáculos. El algoritmo comienza con entornos simples, pero gradualmente se vuelven más difíciles con el tiempo: esto se llama aprendizaje del plan de estudios. Además, aunque cada agente solo está entrenado dentro de un entorno, debe competir regularmente contra otros agentes, en todos los entornos. En cada entorno, se copia al ganador y reemplaza al agente que estaba allí antes. De esta manera, el conocimiento se transfiere regularmente a través de los entornos y se seleccionan los agentes más adaptables. Al final, los agentes son mucho mejores caminantes que los agentes entrenados en una sola tarea, y pueden abordar entornos mucho más difíciles. Por supuesto, este principio también se puede aplicar a otros entornos y tareas. Si está interesado en OEL, asegúrese de consultar el documento Enhanced POET, así como el documento 2021⁠ de DeepMind sobre este tema.

#### TIP

Si quieres aprender más sobre el aprendizaje por refuerzo, echa un vistazo al libro Reinforcement Learning de Phil Winder (O'Reilly).

#### --------------------------------------------------------------------------------------------------------------

Cubrimos muchos temas en este capítulo: gradientes de políticas, cadenas de Markov, procesos de decisión de Markov, aprendizaje Q, aprendizaje Q aproximado y aprendizaje Q profundo y sus variantes principales (objetivos fijos de valor Q, DQN doble, DQN de duelo y reproducción de la experiencia priorizada), y finalmente echamos un vistazo rápido a algunos otros algoritmos populares. El aprendizaje de refuerzo es un campo enorme y emocionante, con nuevas ideas y algoritmos que aparecen todos los días, así que espero que este capítulo haya despertado tu curiosidad: ¡hay todo un mundo por explorar!

# Ejercicios

1. ¿Cómo definirías el aprendizaje por refuerzo? ¿De qué se diferencia del aprendizaje regular supervisado o no supervisado?

2. ¿Se te ocurren tres posibles aplicaciones de RL que no se mencionaron en este capítulo? Para cada uno de ellos, ¿cuál es el medio ambiente? ¿Cuál es el agente? ¿Cuáles son algunas de las acciones posibles? ¿Cuáles son las recompensas?

3. ¿Cuál es el factor de descuento? ¿Puede cambiar la póliza óptima si modifica el factor de descuento?

4. ¿Cómo se mide el rendimiento de un agente de aprendizaje de refuerzo?

5. ¿Cuál es el problema de la asignación de crédito? ¿Cuándo ocurre? ¿Cómo puedes aliviarlo?

6. ¿Qué sentido tiene usar un búfer de reproducción?

7. ¿Qué es un algoritmo RL fuera de la política?

8. Utilice gradientes de políticas para resolver el entorno LunarLander-v2 de OpenAI Gym.

9. Usa un doble duelo DQN para entrenar a un agente que pueda alcanzar un nivel sobrehumano en el famoso juego Atari Breakout (""ALE/Breakout-v5" Las observaciones son imágenes. Para simplificar la tarea, debe convertirlos a escala de grises (es decir, promedio sobre el eje de los canales) y luego recortarlos y muestrearlos a la baja, para que sean lo suficientemente grandes como para jugar, pero no más. Una imagen individual no te dice en qué dirección van la pelota y las paletas, por lo que debes fusionar dos o tres imágenes consecutivas para formar cada estado. Por último, el DQN debe estar compuesto principalmente por capas convolucionales.

10. Si tienes alrededor de 100 dólares de sobra, puedes comprar una Raspberry Pi 3 más algunos componentes de robótica baratos, instalar TensorFlow en el Pi, ¡y irte loco! Por ejemplo, echa un vistazo a esta divertida publicación de Lukas Biewald, o echa un vistazo a GoPiGo o BrickPi. Comience con objetivos simples, como hacer que el robot se dé la vuelta para encontrar el ángulo más brillante (si tiene un sensor de luz) o el objeto más cercano (si tiene un sensor de sonar), y muévase en esa dirección. Entonces puedes empezar a usar el aprendizaje profundo: por ejemplo, si el robot tiene una cámara, puedes intentar implementar un algoritmo de detección de objetos para que detecte a las personas y se mueva hacia ellas. También puedes intentar usar RL para que el agente aprenda por su cuenta cómo usar los motores para lograr ese objetivo. ¡Diviértete!

Las soluciones a estos ejercicios están disponibles al final del cuaderno de este capítulo, en https://homl.info/colab3.